🎯Linear regression is a fundamental statistical technique used to predict the relationship between a dependent variable and one or more independent variables. It assumes a linear relationship between the variables, enabling us to estimate the value of the dependent variable based on the independent variables. In the case of predicting car fuel efficiency, linear regression proves to be an effective tool.
🔍 Here is a step-by-step solution on how I built the model:
1️⃣ Data Collection: The first step involved gathering the Car Auto MPG dataset, which contains information about various car attributes such as horsepower, weight, cylinders, and miles per gallon (MPG).
2️⃣ Data Preprocessing: I performed a comprehensive analysis of the dataset, checking for missing values, outliers, and inconsistencies. Then, I conducted feature engineering to select relevant independent variables and prepared the data for model training.
3️⃣ Splitting the Data: To evaluate the performance of the model accurately, I divided the dataset into a training set and a testing set. The training set was used to train the model, while the testing set helped assess its predictive capabilities.
4️⃣ Model Building: With the dataset prepared, I employed linear regression, a supervised learning algorithm, to train the model. This involved finding the optimal coefficients for each independent variable to create a linear equation that best predicts car fuel efficiency.
5️⃣ Model Evaluation: To evaluate the accuracy and robustness of the model, I used various evaluation metrics such as Mean Squared Error (MSE), Root Mean Squared Error (RMSE), and R-squared. These metrics provided insights into how well the model fitted the data.
6️⃣ Predictions: Once the model was trained and evaluated, I utilized it to make predictions on unseen data, i.e., the testing set. Comparing the predicted MPG values with the actual values allowed me to assess the model's performance on new data.
7️⃣ Model Refinement: Finally, I iteratively refined the model by tweaking hyperparameters, trying different feature combinations, or exploring alternative algorithms to enhance its predictive capabilities.

In [ ]:
# importing libraries
import pandas as pd
import numpy as np

# import linear regression machine learning library
from sklearn.linear_model import LinearRegression

# importing ploting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style
plt.style.use('classic')

%matplotlib inline

In [ ]:
# reading the CSV file into pandas dataframe
mpg_df = pd.read_csv('car-mpg.csv')

In [ ]:
# check top few records to get a feel of thr data structure 
mpg_df.head(20)

In [ ]:
# drop the car name column as it is useless for the model
mpg_df = mpg_df.drop('car_name', axis=1)

In [ ]:
mpg_df.head()

In [ ]:
# Replace the numbers in categorical variables with the actual country names in the origin col
mpg_df['origin'] = mpg_df['origin'].replace({1: 'america', 2:'europe', 3:'asia'})

In [ ]:
mpg_df.head()

In [ ]:
# convert categorical variable into dummy/indicator variable. as many columns will be created as distinct values
# this is also known as one hot encoding. thw column name will be america, europe, and asia.... with one hot coding
mpg_df = pd.get_dummies(mpg_df, columns=['origin'])

In [ ]:
mpg_df.head()

In [ ]:
# lets analyze the distribution of the dependent (mpg) column
mpg_df.describe().transpose()

In [ ]:
mpg_df.dtypes

In [ ]:
# Note: HP column is missing the descibe output. That indicates something is not right with that column

In [ ]:
# check if the hp column contains anything other than digits
# run the "isdigit() check on 'hp' column of the mpg_df dataframe. result will be True or False for every row"
# capture the result in temp dataframe and row a frequency count using value_counts()
# there are six records with non digit values in 'hp' column 

temp = pd.DataFrame(mpg_df.hp.str.isdigit())      # if the string is made of digit store True else False in the hp column
# in the temp dataframe

temp[temp['hp'] == False]   # from temp take only those where hp has false

In [ ]:
# on inspecting records number 32, 126, etc we fing "?" in the columns. replace them with "nan"
# replace them with nan and remove the records from the data frame that have "nan" 
mpg_df = mpg_df.replace('?', np.nan)

In [ ]:
# let us see if we can get those records with nan
mpg_df[mpg_df.isnull().any(axis=1)]

In [ ]:
# there are various ways to handle missing values. drop the rows, replace missing values with median values etc

In [ ]:
# of the 398 rows 6 have NaN in the hp column. we will drop those 6 rows. not a good idea under all situation
# note: HP is missing because of the non-numeric values in the column
# mpg_df = mpg_df.dropna()

In [ ]:
# instead of dropping the rows, lets replace the missing values with the median value.
mpg_df.median()

In [ ]:
# replace the missing values in 'hp' with median value of 'hp' :Note, we do not need to specify the column names
# every column's missing values is replaced with that column's median respectively (axis=0 means columnwise )
# mpg_df = mpg_df.fillna(mpg_df.median())

mpg_df = mpg_df.apply(lambda x:x.fillna(x.median()), axis=0)

In [ ]:
mpg_df.dtypes

In [ ]:
mpg_df['hp'] = mpg_df['hp'].astype('float64')   # convering the 'hp' column from object / string type to float

In [ ]:
mpg_df.describe()

In [ ]:
# let us do a correlation analysis among the different dimensions and also each dimension with the dependent dimension
# this is done using scatter matrix function which creates a dashboard reflection useful information about the dimension
# the result can be stored as a .png file and opened in say, paint to get a larger view

mpg_df_attr = mpg_df.iloc[:, 0:10]

#axes = pd.plotting.scatter_matrix(mpg_df_attr)
#plt.tight_layout()
#plt.savefig()

sns.pairplot(mpg_df_attr, diag_kind='kde')   # to plot density curve insted of histogram

#sns.pairplot(mpg_df_attr)       # to plot histogram, the default

### The data distribution across various dimensions except 'Acc' do not look normal
### Close observation between 'mpg' and other attributes indicate the relationship is not really linear
### relation between 'mpg' and 'hp' show hetroscedacity... which will impact model accuracy
### How about 'mpg' vs 'yr' surprising to see a positive relation

In [ ]:
# copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = mpg_df.drop('mpg', axis=1)
X = X.drop({'origin_america', 'origin_asia', 'origin_europe'}, axis=1)

# copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = mpg_df[['mpg']]

In [ ]:
# Let us break the X and y dataframe into training set and test set. For this we will use
# sklearn package's data splitting function which is based on random function

from sklearn.model_selection import train_test_split

In [ ]:
# split X and y into training and test set in 75:25 ratio 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [ ]:
# invoke the LinearRegression function and find the bestfit model on training data

regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

In [ ]:
# let us explore the coefficients for each of the independent attribute

for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

In [ ]:
# Let us check the intercept for the model

intercept = regression_model.intercept_[0]

print("The intercept for our model is {}".format(intercept))

In [ ]:
regression_model.score(X_train, y_train)

In [ ]:
# model score - R2 or coeff of determinant
# R^2 = 1 - RSS / TSS = RegErr / TSS

regression_model.score(X_test, y_test)

In [ ]:
# is OLS a good model? should we building a simple linear model ? check the residuals for each predictor

fig = plt.figure(figsize=(10, 8))
sns.residplot(x=X_test['hp'], y=y_test['mpg'], color='green', lowess=True)

fig = plt.figure(figsize=(10, 8))
sns.residplot(x=X_test['acc'], y=y_test['mpg'], color='green', lowess=True)

In [ ]:
# so the model explains 85% of the variability in Y using X

In [ ]:
# -------------- Using Statmodel library to get R type outputs ----------------- 

In [ ]:
# R^2 is not a reliable metric as it always increases with addition of more attributes even if the attributes have no
# influence on the predicted variable. Instead we use adjusted R^2 which removes the statistical chance that improve R^2
# scikit does not provide a facility for adjusted R^2.... so we use
# what you obtain in R language
# this library expects the X and Y to be given in one single dataframe

data_train = pd.concat([X_train, y_train], axis=1)
data_train.head()

In [ ]:
import statsmodels.formula.api as smf
lm1 = smf.ols(formula= 'mpg ~ cyl+disp+hp+wt+acc+yr+car_type', data = data_train).fit()
lm1.params

In [ ]:
print(lm1.summary())    # inferential statistics

In [ ]:
# let us check the sum of squared errors by predicting value of y for test cases and
# substracting from the actual y for the test cases

mse = np.mean((regression_model.predict(X_test) - y_test)**2)

In [ ]:
# underroot of mean_sq_error is standard deviation i.e. avg varience between predicted and actual

import math

math.sqrt(mse)

In [ ]:
# so there is avg of 3.0 (roundoff) mpg difference from rel mpg on an avg

In [ ]:
# predict milage (mpg) for a set of attributes not in the training or test set

y_pred = regression_model.predict(X_test)

In [ ]:
# since this is regression, plot the predicted y value vs actual y values for the test data
# A good model's prediction will be close to actual leading to high R and R2 values
# plt.rcParam['figure.dpi'] = 500


plt.scatter(y_test['mpg'], y_pred)

In [ ]:
# ------------- ITERATION 2 --------------

In [ ]:
# How do we improve the model ? the R^2 is .844, how do we improve it
# The independent attributes have different units and scales of measurement
# it is always a good practice to scale all the dimensions using z scores or someother method to address the problem of different scales

In [ ]:
from scipy.stats import zscore

mpg_df_scaled = mpg_df.apply(zscore)

In [ ]:
# convert the contents of the dataframe. Check that all the values are now z scores

mpg_df_scaled

In [ ]:
# copy all the predictor variables into X dataframe. Since 'mpg' is dependent variable drop it
X = mpg_df_scaled.drop('mpg', axis=1)
X = X.drop({'origin_america', 'origin_asia', 'origin_europe'}, axis=1)

# copy the 'mpg' column alone into the y dataframe. This is the dependent variable
y = mpg_df_scaled[['mpg']]

In [ ]:
# split X and y into training and test set in 75:25 ratio 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [ ]:
# invoke the LinearRegression function and find the bestfit model on training data

regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

In [ ]:
# let us explore the coefficients for each of the independent attributes

for idx, col_name in enumerate(X_train.columns):
    print("The coefficient for {} is {}".format(col_name, regression_model.coef_[0][idx]))

In [ ]:
intercept = regression_model.intercept_[0]

print("The intercept for our model is {}".format(intercept))

In [ ]:
# Model score - R2 or coeff of determinant
# R^2=1–RSS / TSS

regression_model.score(X_test, y_test)

In [ ]:
# Let us check the sum of squared errors by predicting value of y for training cases and 
# subtracting from the actual y for the training cases

mse = np.mean((regression_model.predict(X_test) - y_test)**2)

In [ ]:
# underroot of mean_sq_error is standard deviation i.e. avg variance between predicted and actual

import math

math.sqrt(mse)

In [ ]:
# predict mileage (mpg) for a set of attributes not in the training or test set
y_pred = regression_model.predict(X_test)

In [ ]:
# Since this is regression, plot the predicted y value vs actual y values for the test data
# A good model's prediction will be close to actual leading to high R and R2 values

plt.scatter(y_test['mpg'], y_pred)